# Library Visualizations
##### Data visualizations of print books in the University of Edinburgh's library

* **Funding:** Edinburgh Futures Institute (EFI)
* **Data Source:** Library and University Collections (L&UC), OCLC MARC Data for Print Books
* **Project Dates:** 17 June 2019 - 27 July 2019
* **Author:** Lucy Havens

Note: to run this notebook please launch in a terminal window with the following command (the default date rate limit is too small to run the notebook properly): *jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000*

## Preparation

Import required libraries

In [1]:
import csv
import pandas as pd
import numpy as np
import re

Load CSV files

In [2]:
# df0 = pd.read_csv("/afs/inf.ed.ac.uk/user/s15/s1545703/Downloads/LibViz_DICE/Data/MARC_data.csv")

# df_tem = pd.read_csv('Data/VizData_ToClean/temporal_MARC_data.csv')   #("/afs/inf.ed.ac.uk/user/s15/s1545703/Downloads/LibViz_DICE/Data/temporal_MARC_data.csv")
df_geo = pd.read_csv('Data/VizData_ToClean/geographic_MARC_data.csv')   #("/afs/inf.ed.ac.uk/user/s15/s1545703/Downloads/LibViz_DICE/Data/geographic_MARC_data.csv")
# df_top = pd.read_csv('Data/VizData_ToClean/topical_MARC_data.csv')   #("/afs/inf.ed.ac.uk/user/s15/s1545703/Downloads/LibViz_DICE/Data/topical_MARC_data.csv")

# df = pd.concat(map(pd.read_csv, glob.glob('Data/BatchToClean/*.csv))

In [3]:
# df.head()
# df_geo.head()
# df_tem.head()
# df_top.head()
# df_geo.columns
df_publ_place = pd.DataFrame()
df_publ_place['publ_place'] = df_geo['publ_place']
df_publ_place['publ_place1'] = df_geo['publ_place1']
df_publ_place.to_csv('Data/VizData_ToClean/publication_place_MARC_data.csv')

In [5]:
df_top.columns

Index(['Unnamed: 0', 'isbn', 'isbn1', 'loc_call_no', 'subject_cat_code',
       'dewey_no', 'personal_name', 'relator', 'uniform_title',
       'uniform_title1', 'title', 'title_cont', 'stat_of_resp',
       'content_type_term', 'content_type_code', 'titles_words_assoc_w_name',
       'title1', 'general_subdivision', 'form_subdivision',
       'general_subdivision1', 'topic_or_geo', 'general_subdivision2',
       'general_subdivision3', 'genre_form_focus', 'genre_form_focus1'],
      dtype='object')

In [3]:
# shape0 = df_0.shape
# print("Rows:", shape0[0])
# print("Columns:", shape0[1])

df_geo_shape = df_geo.shape
print("Rows:", df_geo_shape[0])
print("Columns:", df_geo_shape[1])
print()
df_tem_shape = df_tem.shape
print("Rows:", df_tem_shape[0])
print("Columns:", df_tem_shape[1])
print()
df_top_shape = df_top.shape
print("Rows:", df_top_shape[0])
print("Columns:", df_top_shape[1])

assert df_geo_shape[0] == df_tem_shape[0]
assert df_top_shape[0] == df_tem_shape[0]

Rows: 1366522
Columns: 19

Rows: 1366522
Columns: 19

Rows: 1366522
Columns: 25


Remove empty columns (columns in which every row is NaN, meaning the values are missing)

In [4]:
# # Maximum number of empty values = total number of rows in the dataframe
# total_rows = df0.shape[0]
# col_list0 = list(df0.columns)
# cols_to_drop = [] # create a list of columns to drop
# for col0 in col_list0:
#     if df0[col0].isnull().sum() == total_rows:
#         cols_to_drop += [col0]

# print("Empty Columns:", cols_to_drop)

In [4]:
# df0.drop(['Unnamed: 0'], axis=1, inplace=True) # columns are axis 1, rows are axis 0
# df0.head()

df_geo.drop(['Unnamed: 0'], axis=1, inplace=True) # columns are axis 1, rows are axis 0
# df_geo.head()

# df_tem.drop(['Unnamed: 0'], axis=1, inplace=True) # columns are axis 1, rows are axis 0
# df_tem.head()

df_top.drop(['Unnamed: 0'], axis=1, inplace=True) # columns are axis 1, rows are axis 0
# df_top.tail()

Find records for print books (identified as "text" ("txt") in the MARC data) and remove the rest of the records.

In [5]:
# List of acceptable values identifying print books (modify as needed for different datasets)
print_book_values = ['text','txt','tetx', 'testo','tekst', 'texto']

df_formats = pd.DataFrame()
df_formats["content_type_term"] = df_top["content_type_term"]
df_formats["content_type_code"] = df_top["content_type_code"]

terms = list(df_formats.iloc[:, 0].unique())
codes = list(df_formats.iloc[:, 1].unique())

unacceptable_terms = []
for term in terms:
    term = term.lower()
    term = term.replace(' ','')
    acceptable = False
    for val in print_book_values:
        if val in term:
            acceptable = True
    if not acceptable:
        unacceptable_terms += [term]

unacceptable_codes = []
for code in codes:
    code = code.lower()
    code = code.replace(' ','')
    acceptable = False
    for val in print_book_values:
        if val in code:
            acceptable = True
    if not acceptable:
        unacceptable_codes += [code]

index_list = []
for t in unacceptable_terms:
    rowsToRemove = df_formats[df_formats.iloc[:,0] == t]
    indecesToRemove = rowsToRemove.index
    if len(indecesToRemove) > 0:
        for i in indecesToRemove:
            index_list += [i]
for c in unacceptable_codes:
    rowsToRemove = df_formats[df_formats.iloc[:,1] == c]
    indecesToRemove = rowsToRemove.index
    if len(indecesToRemove) > 0:
        for i in indecesToRemove:
            index_list += [i]
            
print(len(index_list))

69274


### Note on Boolean Operations

We can combine conditions through logical **boolean operations**. Boolean operations are logical constructs that work through the following **boolean operators**: 

1. **AND (`&`)**: selects a row if **all** conditions joined by an `&` sign are true:
    *  e.g. `df.loc[(df['1960'] > 1000) & (df['1960'] < 3000)]` returs all countries with values between 1000 **and** 3000. 
    
* **OR ('|')**: selects a row if **at least one** of the conditions joined by a `|` (say 'pipe') sign are true:
    * e.g. `df.loc[(df['1960'] < 1000) | (df['1960'] > 3000])` returs all countries with values smaller than 1000 **or** with values larger than 3000. 
    
* **NOT(~)**: selects a row if a **condition is not met**. the `~` charater (say 'tilde') has to stand _before_ the condition: 
    *  e.g. `df.loc[~(df['1960'] < 1000)]` returs all countries with values not lower than 1000 (i.e. rows with values higher than 1000, including 1000.) 

You can combine these boolean operations in many ways using parentheses, as in the following example, which returns all countries with values between 1000 and 3000, or countries with values exactly 10000 in 1960.

`df.loc[((df['1960'] > 1000) & (countryData['1960'] < 3000)) | (countryData['1960'] == 10000)]`

In [13]:
# To make sure the acceptable_terms_code list reflects the dataset:
    
# print(df_top['content_type_code'].value_counts())
# print(df_top['content_type_term'].value_counts())

In [6]:
# df0.drop(index_list, inplace=True)
# df_tem.drop(index_list, inplace=True)
df_geo.drop(index_list, inplace=True)
df_top.drop(index_list, inplace=True)

All dataframes should include the same print books (meaning they have the same number of rows).

In [7]:
# print('All MARC data')
# print("Rows:", df0.shape[0])
# print("Columns:", df0.shape[1])
# print()
# print('Temporal MARC data')
# print("Rows:", df_tem.shape[0])
# print("Columns:", df_tem.shape[1])
# print()
print('Geographic MARC data')
print("Rows:", df_geo.shape[0])
print("Columns:", df_geo.shape[1])
print()
print('Topical MARC data')
print("Rows:", df_top.shape[0])
print("Columns:", df_top.shape[1])

# assert df_top.shape[0] == df_tem.shape[0]
# assert df_geo.shape[0] == df_top.shape[0]
# assert df_tem.shape[0] == df0.shape[0]

Geographic MARC data
Rows: 1297311
Columns: 18

Topical MARC data
Rows: 1297311
Columns: 24


## Cleaning

Clean identification data fields (listed in italics)

1. Consolidate repetitive columns
2. Replace missing field values with an empty string or the digit 0 (depending on column's data type)
2. Reconcile different field values from columns representing the same data point (i.e. *title* and *title1*)

*isbn
isbn1
personal_name
uniform_title
real_world_obj_URI
uniform_title1
real_world_obj_URI1
title
title_cont
stat_of_resp
physical_desc
content_type_term
content_type_code
personal_name1
title1
uniform_title1
real_world_obj_URI2*

In [40]:
def consolidateColumns(col0, col1, df):  # pass in two columns to consolidate and the dataframe of those columns
    clean_values = []

    for index,row in df.iterrows():
        value0 = str(row[col0]).lower()
        value1 = str(row[col1]).lower()
        # if there is a value...
        if "Unknown" not in value0:
            value0_list = value0.split(';')
            if "Unknown" not in value1:
                value1_list = value1.split(';')
                for value1 in value1_list:
                    if value1 not in value0_list:
                        value0_list += [value1]

            clean_values += [value0_list]

        # if one value is missing...
        else:
            if "Unknown" not in value1:
                value1_list = value1.split(';')
                clean_values += [value1_list]
        # if both values are missing...
            else:
                clean_values += [[""]]

    return clean_values

#### ISBN

In [11]:
# print(isbns.isnull().sum())
# print(isbn1s.isnull().sum())
# print(df0['isbn'].value_counts())
# print(df0['isbn1'].value_counts())

Unknown                       829361
\\$cNo price                     318
9787106023560                     98
7106023566 (booklet : set)        96
7805691797                        90
7805699941 (set)                  71
7805695326 (set) :                58
\\$cRMBY1.20                      36
\\$cRMBY1.50                      35
\\$cRMBY1.40                      33
7805695326 (set)                  32
9571507695 (set)                  31
\\$cRMBY1.10                      27
\\$cRMBY1.00                      27
\\$cRMBY1.15                      25
\\$cRMBY1.25                      24
2503360009                        24
\\$cNT{dollar}120.00              23
\\$cRMBY1.30                      21
\\$cNo price (pbk)                21
9575432525                        21
\\$cRMBY1.05                      21
\\$c(pbk)                         20
\\$c(pbk.)                        20
\\$cRMBY1.80                      20
7805697418 (set)                  20
\\$cRMBY1.35                      19
\

In [41]:
# isbns = (df0['isbn'])
# isbn1s = (df0['isbn1'])

isbns = (df_tem['isbn'])
isbn1s = (df_tem['isbn1'])

# isbns = (df_geo['isbn'])
# isbn1s = (df_geo['isbn1'])

# isbns = (df_top['isbn'])
# isbn1s = (df_top['isbn1'])

isbns_list = list(isbns)
isbn1s_list = list(isbn1s)
assert len(isbns_list) == len(isbn1s_list) # confirm isbns and isbn1s are the same length

In [42]:
clean_isbns = consolidateColumns('isbn','isbn1',df_tem)
assert len(clean_isbns) == len(list(df_tem["isbn"]))

In [44]:
def validISBNs(clean_values_list):
    # remove words and abbreviations from list of isbns
    cleaner_isbns = []
    for isbn_list in clean_isbns:
        new_list = []
        for isbn in isbn_list:
            test0 = re.findall("\d+[\D?]$",isbn)
            test1 = re.findall("\d+",isbn)
            if (len(test0) > 0):
                new_list += test0
            elif (len(test1) > 0):
                new_list += test1
            else:
                new_list += ""

        cleaner_isbns += [new_list]

    # Each isbn value should be at least 10 characters in length
    for isbn_list in cleaner_isbns:
        for isbn in isbn_list:
            if (len(isbn) < 10):
                isbn_list.remove(isbn)
    
    return cleaner_isbns

cleaner_isbns = validISBNs(clean_isbns)
assert len(cleaner_isbns) == len(list(df_tem["isbn"]))

In [54]:
# df0_clean = pd.DataFrame()
# df0_clean["isbn"] = cleaner_isbns
# df0_clean.tail()

df_tem_clean = pd.DataFrame()
df_tem_clean["isbn"] = cleaner_isbns
df_tem_clean.tail()

,isbn
1139301,[3100381343]
1139302,[091280405x]
1139303,"[2222027888, 2222027888]"
1139304,"[0404617220, 0404617220]"
1139305,[0824090470]


#### Author
Time permitting: see if author names can by identified in statements of responsibility

In [55]:
# personal_name, stat_of_resp, personal_name1

personal_names = list(df_tem['personal_name'])
statements_of_responsibility = list(df_tem['stat_of_resp'])
unmatched_indeces = []
maxI = len(personal_names)
i = 0
while i < maxI:
    names = str(personal_names[i])
    alpha_list = re.findall('\w+',names)
    statement = str(statements_of_responsibility[i])
    is_in = False
    for name in alpha_list: 
        if name in statement:
            is_in = True
    if not is_in:
        unmatched_indeces += [i]
    i += 1
print(len(unmatched_indeces))

424279


In [46]:
# Use the code below when working with smaller, test datasets
# for i in unmatched_indeces:
#     print(personal_names[i], "|", statements_of_responsibility[i])

In [56]:
# new_names = []
# for item in personal_names:
#     name = str(item)
#     if name == 'Unknown':
#         new_names += ['Unknown']
#     else:
#         m = n.strip(',')
#         new_names += [m]

# assert len(new_names) == len(personal_names)

In [60]:
df_tem_clean["author"] = list(df_tem["personal_name"])
df_tem_clean.tail()

,isbn,author
1139301,[3100381343],"Kafka, Franz,"
1139302,[091280405x],"Mezzatesta, Michael P."
1139303,"[2222027888, 2222027888]",Unknown
1139304,"[0404617220, 0404617220]","Land, Stephen K."
1139305,[0824090470],"Leistritz, Larry,"


#### Publication Date

In [23]:
# print(df0['date_of_work'].isnull().sum())
# print(df0['publ_date'].isnull().sum())
# print(df0['publ_date1'].isnull().sum())
# print(df0['time_period_of_content'].isnull().sum())
# print(df0['geo_subdivision'].isnull().sum())
# print(df0['dates_assoc_w_name'].isnull().sum())
# print(df0['chrono_subdivision'].isnull().sum())
# print(df0['chrono_subdivision1'].isnull().sum())
# print(df0['chrono_subdivision2'].isnull().sum())

396
6
396
397
396
337
368
378
397


Use 'publ_date' column (as it has the fewest empty values in the test dataset).

In [79]:
# INPUT: string date range (contains '-')
# OUTPUT: list with the minimum and maximum dates as 4-digit strings
# Example 1: '1928-29' --> [1928, 1929]
# Example 2: '1830-5'  --> [1830, 1835]
def dateRangeToList(date_range):
    new_dates = []
    date_list = date_range.split('-')
    for d in date_list:
        d = d.strip('.')
        d = d.strip(',')
        if len(d) == 2:
            first_two = date_list[0][0] + date_list[0][1]
            new_d = first_two + d
            new_dates += [int(new_d)]
        elif len(d) == 1:
            first_three = date_list[0][0] + date_list[0][1] + date_list[0][2]
            new_d = first_three + d
            new_dates += [int(new_d)]
        elif len(d) == 4:
            new_dates += [int(d)]
        else:
            print(d)

    return new_dates

# From: www.oreilly.com/library/view/python-cookbook/0596001673/ch03s24.html
def int_to_roman(input):
    if not isinstance(input, type(1)):
        return "Expected type integer"
    if not 0 < input < 4000:
        return "Argument must be between 1 and 3999"
    ints = (1000, 900,  500, 400, 100,  90, 50,  40, 10,  9,   5,  4,   1)
    nums = ('M',  'CM', 'D', 'CD','C', 'XC','L','XL','X','IX','V','IV','I')
    result = []
    for i in range(len(ints)):
        count = int(input / ints[i])
        result.append(nums[i] * count)
        input -= ints[i] * count
    return ''.join(result)

# INPUT: string of capital letters (roman numerals)
# OUTPUT: 4-digit string (year)
def romanToDigits(roman_numerals):
    # Remove any punctuation and spaces from the inputted string
    no_spaces = roman_numerals.replace(" ","")
    input = no_spaces.replace(".","")
    
    # From: www.oreilly.com/library/view/python-cookbook/0596001673/ch03s24.html
    if not isinstance(input, type("")):
        return "Expected string"
    input = input.upper(  )
    nums = {'M':1000, 'D':500, 'C':100, 'L':50, 'X':10, 'V':5, 'I':1}
    sum = 0
    for i in range(len(input)):
        try:
            value = nums[input[i]]
            # If the next place holds a larger number, this value is negative
            if i+1 < len(input) and nums[input[i+1]] > value:
                sum -= value
            else: sum += value
        except KeyError:
            return "0"
    # easiest test for validity...
    if int_to_roman(sum) == input:
        return sum
    else:
        return "0"
    

# INPUT: century string (matches regex pattern '\d{2}\w{2}')
# Examples: '20th century', '21st'
# OUTPUT: list of minimum and maximum years as 4-digit strings
# Examples: ['1900','1999'], ['1900','2099']
def centuryToYears(c):
    digits = re.findall('\d{2}',c)
    years = []
    for century in digits:
        four_digits = int(century + '00')
        min_year = four_digits - 100
        max_year = four_digits - 1
    
    return [int(min_year),int(max_year)]
    

# INPUT: date string
# OUTPUT: 4 digit year as integer (earliest if multiple) UNLESS no date provided, in which case 0 is returned
def cleanDate(date):
    # If no date is provided...
    if (str(date) == 'Unknown'):
        return 0
    else:
        # If a year or range of years is provided...
        date_range = re.findall('\d{4}', date)
        if len(date_range) == 1:
            return int(date_range[0])
        elif len(date_range) > 1:
            # Turn date strings to integers
            min_date = int(date_range[0])
            for r in date_range:
                r = int(r)
                if r < min_date:
                    min_date = r
            return min_date

        # When len(date_range) == 0...
        else: 
            centuries = re.findall('\d{2}\w{2}', date)
            # If a century is provided, return the first year of that century
            if len(centuries) > 0:
                for c in centuries:
                    year_list = centuryToYears(c)
                    return min(year_list)

            # When len(centuries) == 0...
            else:
                # If roman numerals are provided, return the corresponding 4-digit year
                return romanToDigits(date)

In [103]:
publ_dates = list(df_tem['publ_date'])
new_publ_dates = []
for d in publ_dates:
    new_d = cleanDate(d)
    if int(new_d) > 2019:  # If dates are in the future they must be a typo in the catalog
        new_d = 0
    new_publ_dates += [int(new_d)]

assert len(publ_dates) == len(new_publ_dates)

In [82]:
# Manual check
maxI = 20
i = 0
while i < maxI:
    print(i, ":", df_tem.iloc[i].publ_date, "|", publ_dates[i], "|", new_publ_dates[i])
    i += 1

0 : 1828. | 1828. | 1828
1 : 1817. | 1817. | 1817
2 : 2007. | 2007. | 2007
3 : 2008. | 2008. | 2008
4 : 2007. | 2007. | 2007
5 : 2008. | 2008. | 2008
6 : 1838-1839. | 1838-1839. | 1838
7 : MDCCCXVII [1817] | MDCCCXVII [1817] | 1817
8 : 1878. | 1878. | 1878
9 : [2003], ©2003. | [2003], ©2003. | 2003
10 : [2004], ©2004. | [2004], ©2004. | 2004
11 : MDCCCXVII [1817] | MDCCCXVII [1817] | 1817
12 : 1967. | 1967. | 1967
13 : MDCCCXVII [1817] | MDCCCXVII [1817] | 1817
14 : MDCCCXVII [1817] | MDCCCXVII [1817] | 1817
15 : 1986. | 1986. | 1986
16 : 1967. | 1967. | 1967
17 : 2008. | 2008. | 2008
18 : 1986. | 1986. | 1986
19 : MDCCCXVII [1817] | MDCCCXVII [1817] | 1817


In [84]:
# manual check
i = 0
maxI = len(new_publ_dates)
bad_indeces = []
while i < maxI:
    if (new_publ_dates[i] == "Unknown") or (int(new_publ_dates[i]) > 2019):
        bad_indeces += [i]
    
    i += 1
print("Rows to fix:", len(bad_indeces), "(",(len(bad_indeces)/len(new_publ_dates))*100,"%)")

Rows to fix: 0 ( 0.0 %)


In [114]:
# df_tem_clean["publication_date"] = new_publ_dates
# df_tem_clean.tail()
# new_df_tem = pd.DataFrame()
# new_df_tem['publ_date'] = new_publ_dates
publ_counts = new_df_tem['publ_date'].value_counts()
# publ_counts.sort
df_pub = pd.DataFrame(publ_counts)
df_pub.head()

,publ_date
0,123378
1996,19679
1997,19418
1995,19152
1998,19059


In [115]:
df_pub.to_csv('Data/VizData/print_book_yearly_publ_totals.csv')

#### Title
Time permitting: clean!!!

In [82]:
# INPUT: a dataframe and two MARC fields (dataframe columns) with values of type string
# OUTPUT: a list of strings, where each string is a value from col2 appended to a value from col1 in the corresponding row
def combineColValues(df,col1,col2):
    titles = list(df[col1])
    titles_cont = list(df[col2])
    full_title_list = []
    i = 0
    maxI = len(titles)
    while i < maxI:
        t1 = titles[i]
        t2 = titles_cont[i]
        if t2 != 'Unknown':
            new_title = t1 + t2

        else:
            new_title = t1
        
        new_title.strip('/')
        new_title.strip()
        full_title_list.append(new_title)
            
        i += 1
        
    return full_title_list
    
full_titles = combineColValues(df_tem,'title','title_cont')

df_tem_clean["title"] = full_titles
df_tem_clean.head()

,isbn,author,publication_date,title
0,[],"Apollonius,",1828,Apollonii Rhodii Argonautica ::Ad fidem libror...
1,"[9781403976277, 9781403976277]","Ebenstein, Alan O.",2007,Milton Friedman :a biography /
2,"[9780230007895, 9780230007895]","Szenberg, Michael.",2008,Franco Modigliani :a mind that never rests /
3,"[1403996237, 1403996237]","Davidson, Paul,",2007,John Maynard Keynes /10$aJohn Maynard Keynes /...
4,"[1842143050, 1842143050]",Unknown,2008,Atlas of contraception /00$aAtlas of contracep...


Export a CSV file of the number of books published in each year.

In [100]:
books_per_year = pd.DataFrame(df_tem_clean['publication_date'].value_counts().reset_index().values, columns=["publication_date","total_print_books"])
# books_per_year.sort_values(by="publication_date", axis=1, ascending=True)
books_per_year
books_per_year.to_csv('Data/VizData/UoELibraryBooksPublishedPerYear.csv')

#### Publication Place

In [29]:
# print(df0.publ_place.isnull().sum())
# print(df0.publ_place1.isnull().sum())
# print(df0.geographic_area_code.isnull().sum())
# print(df0.geo_subdivision.isnull().sum())
# print(df0.geo_subdivision1.isnull().sum())
# print(df0.geo_subdivision2.isnull().sum())
# print(df0.geo_name.isnull().sum())
# print(df0.publisher.isnull().sum())
# print(df0.publisher1.isnull().sum())
# print(df0.text_language.isnull().sum())

6
396
285
396
294
394
346
7
396
359


Use 'publ_place' (as it has the fewest empty values in the test dataset).

In [7]:
# publ_places = list(df0.publ_place)
publ_places = df_geo.publ_place

In [8]:
clean_publ_places = []

for p in publ_places:
    p = str(p)
    if p == 'nan':
        clean_publ_places += [[]]
    else:
        if (p == "Newcastle-Ont.-Tyne"):  # manual fix for typo
            p = "Newcastle-on-Tyne"
        places = re.findall('[^\W\d]+[a-zA-Z\-a-zA-Z]*[ öüèéí.]*[a-zA-Z\-a-zA-Z]*[ ]*[^\W\d]+[.]*',p)  # chars to consider: áàåâæçéèîñöœûü
        clean_places = []
        for new_p in places:
            clean = new_p.strip() # remove trailing whitespace
            if len(clean) > 0:
                clean_places += [clean]
        clean_publ_places += [clean_places]

assert len(publ_places) == len(clean_publ_places)
# clean_publ_places

In [9]:
# df0_clean["publication_place"] = clean_publ_places
df_geo_clean = pd.DataFrame()
df_geo_clean["publication_place"] = clean_publ_places
df_geo_clean.head()

,publication_place
0,[Lipsiae]
1,[Edinburgi]
2,[New York]
3,[Basingstoke]
4,[Basingstoke]


In [10]:
all_publ_places = list(df_geo_clean['publication_place'])
publ_place_counts = {(all_publ_places[0][0]):1}
for place_list in all_publ_places:
    for place in place_list:
        if place in publ_place_counts:
            publ_place_counts[place] += 1
        else:
            publ_place_counts[place] = 1
print(len(publ_place_counts))


# d = {}
# d[1] = 1
# if 2 in d:
#     d[2] += 1
# else:
#     d[2] = 1
# print(d)

26711


In [11]:
# df0_clean
df_geo_counts = pd.DataFrame()
df_geo_counts['Publication Place'] = list(publ_place_counts.keys())
df_geo_counts['Total Books'] = list(publ_place_counts.values())
df_geo_counts.head()

,Publication Place,Total Books
0,Lipsiae,2277
1,Edinburgi,4237
2,New York,83233
3,Basingstoke,5289
4,Boca Raton,516


In [12]:
df_geo_counts.to_csv('Data/VizData/books_per_publication_place.csv')

## Adding Associated Data

#### Subject Classification

In [23]:
# print(df0['loc_call_no'].isnull().sum())       # 159
# print(df0['subject_cat_code'].isnull().sum())
# print(df0['dewey_no'].isnull().sum())          # 304
# print(df0['titles_words_assoc_w_name'].isnull().sum())
# print(df0['general_subdivision'].isnull().sum())
# print(df0['form_subdivision'].isnull().sum())
# print(df0['general_subdivision1'].isnull().sum())
# print(df0['topic_or_geo'].isnull().sum())      # 144
# print(df0['general_subdivision2'].isnull().sum())
# print(df0['general_subdivision3'].isnull().sum())
# print(df0['genre_form_focus'].isnull().sum())
# print(df0['genre_form_focus1'].isnull().sum())
# print(df0['personal_name1'].isnull().sum())
# print(df0['title1'].isnull().sum())
# print()
# print(df0['uniform_title1'].isnull().sum())
# print()
# print(df0['real_world_obj_URI2'].isnull().sum())

In [24]:
# loc_call_nos = list(df0.loc_call_no)
# topic_or_geo = list(df0.topic_or_geo)
loc_call_nos = list(df_top.loc_call_no)
topic_or_geo = list(df_top.topic_or_geo)

In [25]:
loc_call_nos

['PA3872',
 'Unknown',
 'HB119.F84',
 'HB109.M63',
 'HB103.K47',
 'RG136',
 'PA3443',
 'Unknown',
 'PA3872.Z4',
 'HB501',
 'HN13',
 'Unknown',
 'PA4035.N5',
 'Unknown',
 'Unknown',
 'PL852.O8',
 'PA4035.N5',
 'JZ1480.A54',
 'Unknown',
 'Unknown',
 'PN1978.J3',
 'PL787.T3',
 'KJC1083',
 'PA3872.Z4',
 'KJE947',
 'DS835',
 'DS894.59.A352',
 'DS894.79.O372',
 'DS894.39.F832',
 'DS894.39.M592',
 'DS894.79.H562',
 'DS894.99.O365',
 'PN1031',
 'DS894.59.G532',
 'DS896.38',
 'KJC1854',
 'DC430',
 'DS894.69.K952',
 'DS894.69.S452',
 'DS895.G5',
 'DC404',
 'DS894.99.O365',
 'Unknown',
 'DS895.I77',
 'DS855',
 'Unknown',
 'PR6045.I55',
 'Unknown',
 'Unknown',
 'PL758.I5',
 'PL758.I5',
 'PL758.I5',
 'PL758.I5',
 'KJC1826',
 'PL758.I5',
 'PL758.I5',
 'PL758.I5',
 'Unknown',
 'KJC1886',
 'Unknown',
 'PA3872.Z4',
 'Unknown',
 'PA4216',
 'PA3082',
 'PS163',
 'PA3872.Z4',
 'PA3061',
 'Unknown',
 'B765.A21',
 'P306.8.A35',
 'DP172',
 'PA4253.O65',
 'PA6766',
 'PA4216',
 'Unknown',
 'Unknown',
 'PA4216',

Obtain the LCC main class of a record (in this case, a printed book) from the Library of Congress Call Number

*Based on the Library of Congress Classification (LCC) Outline: https://www.loc.gov/catdir/cpso/lcco/*

In [28]:
lcc_dict = {'A':'GENERAL WORKS','B':'PHILOSOPHY. PSYCHOLOGY. RELIGION','C':'AUXILIARY SCIENCES OF HISTORY',
            'D':'WORLD HISTORY AND HISTORY OF EUROPE, ASIA, AFRICA, AUSTRALIA, NEW ZEALAND, ETC.',
            'E':'HISTORY OF THE AMERICAS','F':'HISTORY OF THE AMERICAS','G':'GEOGRAPHY. ANTHROPOLOGY. RECREATION',
            'H':'SOCIAL SCIENCES','J':'POLITICAL SCIENCE','K':'LAW','L':'EDUCATION','M':'MUSIC AND BOOKS ON MUSIC',
            'N':'FINE ARTS','P':'LANGUAGE AND LITERATURE','Q':'SCIENCE','R':'MEDICINE','S':'AGRICULTURE',
            'T':'TECHNOLOGY','U':'MILITARY SCIENCE','V':'NAVAL SCIENCE','Z':'BIBLIOGRAPHY. LIBRARY SCIENCE. INFORMATION RESOURCES (GENERAL)'}

lcc_main_class_letters = lcc_dict.keys()

def validLCC(call_number, lcc_main_class_letters):
    first_char = str(call_number)[0]
    if first_char in lcc_main_class_letters:
        return True
    else:
        return False

lcc_main_class_list = []
for call_no in loc_call_nos:
    call_no = str(call_no)
    if (call_no == 'nan') or (not validLCC(call_no, lcc_main_class_letters)):
        lcc_main_class_list += ["UNKNOWN"]
    else:
        first_letter = call_no[0]
        main_class = lcc_dict[first_letter]
        lcc_main_class_list += [main_class]
        
assert len(lcc_main_class_list) == len(loc_call_nos)
print(len(lcc_main_class_list))

1297311


In [29]:
# df0_clean['lcc_no'] = loc_call_nos
# df0_clean['lcc_main_class'] = lcc_main_class_list
#df0_clean
df_top_clean = pd.DataFrame()
df_top_clean['lcc_no'] = loc_call_nos
df_top_clean['lcc_main_class'] = lcc_main_class_list
df_top_clean.tail()

,lcc_no,lcc_main_class
1297306,PT2621.A26,LANGUAGE AND LITERATURE
1297307,NB623.B5,FINE ARTS
1297308,HF3756.5,SOCIAL SCIENCES
1297309,P81.G7,LANGUAGE AND LITERATURE
1297310,HD47.4,SOCIAL SCIENCES


In [36]:
print(len(pd.unique(topic_or_geo)))

68348


In [37]:
new_topic_or_geo = []
for terms in topic_or_geo:
    terms = str(terms)
    if terms == 'nan':
        new_terms = []
    else:
        terms = terms.replace('.','')
        new_terms = list(set(terms.split(';')))  # set is list of unique elements (no duplicates)
    new_topic_or_geo += [new_terms]

assert len(new_topic_or_geo) == len(topic_or_geo)

In [39]:
# new_topic_or_geo

In [52]:
# df0_clean['topic_or_geo'] = new_topic_or_geo
#df0_clean

# df_geo_clean['topic_or_geo'] = new_topic_or_geo
# df_geo_clean.head()

df_top_clean['topic_or_geo'] = new_topic_or_geo
df_top_clean.head()

,lcc_no,lcc_main_class,ddc_no,ddc_main_class,topic_or_geo
0,PA3872,LANGUAGE AND LITERATURE,Unknown,Unknown,[Argonauts (Greek mythology)]
1,Unknown,MILITARY SCIENCE,Unknown,Unknown,[Hepatitis]
2,HB119.F84,SOCIAL SCIENCES,Unknown,Unknown,[Economists]
3,HB109.M63,SOCIAL SCIENCES,B,Unknown,[Keynesian economics]
4,HB103.K47,SOCIAL SCIENCES,330.156,Social sciences,[Economists]


Obtain the DDC main class of a record (in this case, a printed book) from the Dewey Decimal Classification number

*Based on the DDC23 summaries: https://www.oclc.org/content/dam/oclc/dewey/ddc23-summaries.pdf*

In [47]:
dewey_nos = list(df_top.dewey_no)

# INPUT: Dewey Decimal Classification number (string)
# OUTPUT: Dewey Decimal Classification main class name (string)
def DdcClassName(ddc_number):

    ddc_dict = {'000':'Computer science, information & general works',
            '100':'Philosophy & psychology','200':'Religion','300':'Social sciences',
            '400':'Language','500':'Science','600':'Technology','700':'Arts & recreation',
            '800':'Literature','900':'History & geography'}
    
    if str(ddc_number) == 'nan':
        return "Unknown"
    else:
        digits = re.findall('\d{1}',ddc_number)
        if (len(digits) > 0):
            first_digit = digits[0]
            main_class_no = first_digit + '00'
        else:
            return "Unknown"
        return ddc_dict[main_class_no]


ddc_main_class_list = []
for call_no in dewey_nos:
    ddc_main_class_list += [DdcClassName(str(call_no))]
        
assert len(ddc_main_class_list) == len(dewey_nos)
ddc_main_class_list

['Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Social sciences',
 'Technology',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Social sciences',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Literature',
 'Unknown',
 'Social sciences',
 'Literature',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Social sciences',
 'Unknown',
 'Unknown',
 'Unknown',
 'Social sciences',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Arts & recreation',
 'Unknown',
 'Unknown',
 'Unknown',
 'Literature',
 'Literature',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Language',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Computer

In [53]:
# df0_clean['ddc_no'] = dewey_nos
# df0_clean['ddc_main_class'] = ddc_main_class_list
# df0_clean

df_top_clean['ddc_no'] = dewey_nos
df_top_clean['ddc_main_class'] = ddc_main_class_list

df_top_clean

df_top_clean.head()

,lcc_no,lcc_main_class,ddc_no,ddc_main_class,topic_or_geo
0,PA3872,LANGUAGE AND LITERATURE,Unknown,Unknown,[Argonauts (Greek mythology)]
1,Unknown,MILITARY SCIENCE,Unknown,Unknown,[Hepatitis]
2,HB119.F84,SOCIAL SCIENCES,Unknown,Unknown,[Economists]
3,HB109.M63,SOCIAL SCIENCES,B,Unknown,[Keynesian economics]
4,HB103.K47,SOCIAL SCIENCES,330.156,Social sciences,[Economists]


Export a CSV of the cleaned topical dataframe.

In [54]:
df_top_clean.to_csv('Data/VizData/topical_print_book_data.csv')

Determine how many records have no subject classification information (meaning no topical term nor geographic name, no Library of Congress call number, and no Dewey Decimal call number)

In [51]:
no_subjects = []
for index,row in df0_clean.iterrows():
    if (len(row['topic_or_geo']) == 0) and (row['lcc_main_class'] == "UNKNOWN") and (row['ddc_main_class'] == "Unknown"):
        no_subjects.append(index)

print(len(no_subjects),"records do not have subject information:")
print(no_subjects)

#### Publication Place: Countries and Coordinates

In [48]:
cities_countries = pd.read_json("Data/MapData/ne_50m_populated_places/ne_50m_populated_places.json",typ='frame')
print(cities_countries.head())
print(cities_countries.shape)

                                            features               type
0  {'type': 'Feature', 'geometry': {'type': 'Poin...  FeatureCollection
1  {'type': 'Feature', 'geometry': {'type': 'Poin...  FeatureCollection
2  {'type': 'Feature', 'geometry': {'type': 'Poin...  FeatureCollection
3  {'type': 'Feature', 'geometry': {'type': 'Poin...  FeatureCollection
4  {'type': 'Feature', 'geometry': {'type': 'Poin...  FeatureCollection
(1249, 2)


In [49]:
print("City:", cities_countries.iloc[0].features['properties']['NAME'])             # NAME = city name
print("Country:", cities_countries.iloc[0].features['properties']['ADM0NAME'])      # SOV0NAME = country name
print("Coordinates:", cities_countries.iloc[0].features['geometry']['coordinates'])  # coordinates = latitude,longitude

City: Bombo
Country: Uganda
Coordinates: [32.533299524864844, 0.583299105614628]


Create CSV files of cities, their countries and their coordinates, and the number of records published in that city

In [50]:
# First create one list each for worldwide cities, worldwide countries, and worldwide city coordinates, and
# a list of cities per country contained in a list of all countries worldwide

cities_list = []
countries_list = []
cities_per_country = []
city_coords_list = []
maxI = cities_countries.shape[0]
i = 0
country = cities_countries.iloc[0].features['properties']['ADM0NAME']
country_cities = []
while i < maxI:
    new_country = cities_countries.iloc[i].features['properties']['ADM0NAME']
    new_city = cities_countries.iloc[i].features['properties']['NAME']
    
    # Create a list of cities per country
    if country == new_country:
        country_cities.append(new_city)
    else:
        cities_per_country.append([country,country_cities])
        # Create a list of unique countries (non-repeating)
        countries_list.append(country)
        country = new_country
    
    # Create a list of unique cities (non-repeating)
    cities_list.append(new_city)
    # Create a list of unique coordinates for each city (non-repeating)
    city_coords_list.append(cities_countries.iloc[i].features['geometry']['coordinates'])
    
    i += 1

assert len(city_coords_list) == len(cities_list)

# print(cities_per_country)

In [51]:
# Second create a list of the total number of books (a.k.a. records or dataframe rows) published in each city

publ_cities = []
publ_cities_lists = df0_clean['publication_place']
for cities in publ_cities_lists:
    for city in cities:
        if '.' not in city:       # states are abbreviated with periods (.)
            publ_cities.append(city)

#publ_cities  # Note that some items in this list are actually countries

In [52]:
# countries_list

In [53]:
# INPUT: [first] a list of cities worldwide (strings) and [second] a list of cities to count
# OUTPUT: list of the total times each city from the second list appears for every city in the first list
def countCities(world_cities,to_count):
    city_counts = []
    for world_city in world_cities:
        count = 0
        for city in to_count:
            if world_city in city:
                count += 1
        city_counts.append(count)
    
    return city_counts

city_totals = countCities(cities_list,publ_cities)
assert len(city_totals) == len(cities_list)

In [54]:
# Third create a list of the total number of books (a.k.a. records or dataframe rows) published in each country

# INPUT: [first] a list of countries worldwide (strings) and their corresponding cities (lists of strings) 
#        and [second] a list of cities to count
# OUTPUT: list of the total times each city from the second list appears for every city in the first list
def countCountriesByCity(world_cities_per_country,to_count):
    country_totals = []
    for world_country in world_cities_per_country:
        count = 0
        country = world_country[0]
        cities = world_country[0]
        for city in to_count:
            if city in cities:
                count += 1
        country_totals.append(count)
    
    return country_totals

country_totals = countCountriesByCity(cities_per_country,publ_cities)
assert len(country_totals) == len(countries_list)